# LLM-Symbolic Explanation Refinement

Given a premise sentence $p_i$, a hypothesis sentence $h_i$, and an explanation $E_i$ consisting of a set of facts $\{f_1,f_2,...,f_n\}$, the explanation $E_i$ is considered logically valid if and only if the entailment $p_i \cup E_i \models h_i$ holds. This entailment is considered verifiable if $\{p_i, E_i, h_i\}$ can be translated into a set of logical formulae $\Phi$ that compose a theory $\Theta$. The validity of the theory $\Theta$ is subsequently determined by a theorem prover, verifying whether $\Theta \vDash \psi$, where $\psi$ represents a logical consequence derived from the logical form of $h_i$.

<img src="figures/nesy_refinement.png">

<img src="figures/nesy_refinement_results.png">

## Explanation Refinement (e-SNLI)

Premise: A couple playing with a little boy on the beach.

Hypothesis: "A couple are playing with a young child outside.

Annotated Explanation: little boy is a young child.

In [3]:
from refinement.refinement_model import RefinementModel
from generation.generative_model import GPT
from critique.critique_model import IsabelleSolver
import yaml

# data name is default to example_data
# premise and explanation are default to None
# explanation sentences need to be seperate by '\n'
data_name = 'esnli_0'
premise = "A couple playing with a little boy on the beach."
hypothesis = "A couple are playing with a young child outside."
explanation = "little boy is a young child."


llm = GPT('gpt-4o-mini', api_key)

isabelle_solver = IsabelleSolver(
    generative_model=llm,
    isabelle_session = 'HOL',
    theory_name = data_name
)

# prompts can be modifed, but defaults to following dictionary
# prompt_dict = {
#     'generate explanation': 'get_explanation_prompt.txt',
#     'refine logical error no premise': 'refine_logical_error_no_premise_prompt.txt',
#     'refine logical error with premise': 'refine_logical_error_with_premise_prompt.txt'
# }
refinement_model = RefinementModel(
    generative_model=llm,
    critique_model=isabelle_solver,
)

# premise and explanation are optional
# iterative refinement times are set to 10 by default
results = refinement_model.refine(
    hypothesis = hypothesis,
    premise = premise,
    explanation = explanation,
    data_name = data_name,
)

isabelle_solver.shutdown()


================= Iteration Number 0 ==================

Premise: A couple playing with a little boy on the beach.
Hypothesis: A couple playing with a little boy on the beach.
Explanation: 
1. little boy is a young child

-------------- Verification and Refinement -------------------

------------------ 1) Syntactic Parsing ------------------

Hypothesis Sentence: 
1. A couple are playing with a young child outside
Subject: A couple
Verb Phrase: are playing with a young child outside
 - Main Verb: playing
 - Auxiliary Verb: are
Direct Object: a young child
Adverbial Modifier (Location): outside

Explanation Sentence: 
1. little boy is a young child
Subject: little boy
Linking Verb: is
Subject Complement: a young child

Premise Sentence: 
1. A couple playing with a little boy on the beach
Subject: A couple
Verb Phrase: playing with a little boy on the beach
 - Main Verb: playing
Direct Object: a little boy
Adverbial Modifier (Location): on the beach

------------------ 2) Neodavidsonia

In [21]:
print("hypothesis:", results["hypothesis"])
print("premise:", results["premise"])
print("\n------------------ Explanation ----------------------\n")
print(results["explanation"])

hypothesis: A couple are playing with a young child outside.
premise: A couple playing with a little boy on the beach.

------------------ Explanation ----------------------

A little boy is a young child.
Being on the beach implies that the event is outside.


In [22]:
results["critique output"]["proof tactics"]

['Sledgehammering...',
 'cvc4 found a proof...',
 'cvc4: Try this: using assms explanation_1 explanation_2 by blast (1 ms)',
 'vampire found a proof...',
 'vampire: Found duplicate proof',
 'spass found a proof...',
 'spass: Found duplicate proof',
 'zipperposition found a proof...',
 'zipperposition: Found duplicate proof',
 'Done']